<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#A-quick-intro-to-Command" data-toc-modified-id="A-quick-intro-to-Command-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A quick intro to Command</a></span><ul class="toc-item"><li><span><a href="#Example:-Table-Selector-Mini-Language" data-toc-modified-id="Example:-Table-Selector-Mini-Language-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example: Table Selector Mini Language</a></span></li></ul></li></ul></div>

# A quick intro to Command

In [1]:
from verb import *

In a nutshell, you make a str-to-func mapping (or use the default)

In [2]:
import operator as o

func_of_op_str = {  # Note: Order represents precedence!
    '-': o.sub,
    '+': o.add,
    '*': o.mul,
    '/': o.truediv,
}

# You give it a command string

command_str = '1 + 2 - 3 * 4 / 8'
command = Command(command_str, func_of_op_str)

# You execute the command
command()

1.5

You give it a command string

In [3]:
command_str = '1 + 2 - 3 * 4 / 8'
command = Command(command_str, func_of_op_str)

You execute the command

In [4]:
command()

1.5

It may be useful to see what the operation structure looks like

In [5]:
d = command.to_dict()
d

{'-': ({'+': (1, 2)}, {'*': (3, {'/': (4, 8)})})}

In [6]:
# Or if you read better with indents

from functools import partial
import json
from lined import Pipe

print_jdict = Pipe(partial(json.dumps, indent=2), print)  # Note: Only works if your dict is JSON-izable. 

print_jdict(d)

{
  "-": [
    {
      "+": [
        1,
        2
      ]
    },
    {
      "*": [
        3,
        {
          "/": [
            4,
            8
          ]
        }
      ]
    }
  ]
}


That same dict can be used as a parameter to make the same command

In [7]:
command = Command(d, func_of_op_str)
command()

1.5

## Example: Table Selector Mini Language

In [8]:
import operator as o
from typing import Callable, Mapping
from functools import partial

import pandas as pd
from lined import Pipe
from verb import str_to_basic_pyobj, Command


dflt_func_of_op_str_for_table_selection = {  # Note: Order represents precedence!
    '&': o.__and__,
    '==': o.__eq__,
    '<=': o.__le__,
    '>=': o.__ge__,
    '<': o.__lt__,
    '>': o.__gt__,
}


def mk_table_selector(
    table: pd.DataFrame,
    func_of_op_str: Mapping[str, Callable] = dflt_func_of_op_str_for_table_selection
):

    def leaf_processor(x):
        x = str_to_basic_pyobj(x)
        if x in df:
            return df[x]
        return x

    run_command = Pipe(
        partial(
            Command.from_string,
            func_of_op_str=func_of_op_str,
            leaf_processor=leaf_processor
        ),
        lambda f: f(),
        lambda idx: df[idx],
    )

    return run_command

In [9]:
import pandas as pd

# from mongodol.tests import data
# data.nums_and_lans

df = pd.DataFrame(
    [{'source': 'audio', 'bt': 5, 'tt': 7, 'annot': 'cat'},
     {'source': 'audio',
        'bt': 6,
        'tt': 9,
        'annot': 'dog',
        'comments': 'barks and chases cat away'},
        {'source': 'visual', 'bt': 5, 'tt': 8, 'annot': 'cat'},
        {'source': 'visual',
         'bt': 6,
         'tt': 15,
         'annot': 'dog',
         'comments': 'dog remains in view after bark ceases'}]
)
df

,source,bt,tt,annot,comments
0,audio,5,7,cat,NaN
1,audio,6,9,dog,barks and chases cat away
2,visual,5,8,cat,NaN
3,visual,6,15,dog,dog remains in view after bark ceases


In [10]:
run_command = mk_table_selector(df)

In [11]:
run_command('source == audio')

,source,bt,tt,annot,comments
0,audio,5,7,cat,NaN
1,audio,6,9,dog,barks and chases cat away


In [12]:
run_command('tt <= 8')

,source,bt,tt,annot,comments
0,audio,5,7,cat,NaN
2,visual,5,8,cat,NaN


In [13]:
run_command('source == audio & tt <= 8')

,source,bt,tt,annot,comments
0,audio,5,7,cat,NaN
